In [18]:
from IPython.display import display, Markdown  # 在jupyter显示信息的工具

from langchain.chains import RetrievalQA #检索QA链，在文档上进行检索
# from langchain.chat_models import ChatOpenAI #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.indexes import VectorstoreIndexCreator #导入向量存储索引创建器
from langchain.vectorstores import DocArrayInMemorySearch #向量存储
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings  # ollama嵌入

import pandas as pd

In [52]:
# 加载中文数据
file = "../data/product_data.csv"
loader = CSVLoader(file_path=file)
data = loader.load()

# 查看数据
test_data = pd.read_csv(file, skiprows=0)
display(test_data.shape)
display(test_data.head())

(25, 2)

,product_name,description
0,全自动咖啡机,规格:\n大型 - 尺寸：13.8'' x 17.3''。\n中型 - 尺寸：11.5'' ...
1,电动牙刷,规格:\n一般大小 - 高度：9.5''，宽度：1''。\n\n为什么我们热爱它:\n我们的...
2,橙味维生素C泡腾片,规格:\n每盒含有20片。\n\n为什么我们热爱它:\n我们的橙味维生素C泡腾片是快速补充维...
3,无线蓝牙耳机,规格:\n单个耳机尺寸：1.5'' x 1.3''。\n\n为什么我们热爱它:\n这款无线蓝...
4,瑜伽垫,规格:\n尺寸：24'' x 68''。\n\n为什么我们热爱它:\n我们的瑜伽垫拥有出色的...


In [53]:
# 创建 Ollama 嵌入模型实例
embeddings = OllamaEmbeddings(model="qwen2.5")  # 或其他在 Ollama 中安装的模型名称
# 将指定向量存储类,创建完成后，我们将从加载器中调用,通过文档记载器列表加载
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch, embedding=embeddings,  # 添加 embedding 参数
).from_loaders(
    [loader]
)

# 通过指定语言模型、链类型、检索器和我们要打印的详细程度来创建检索QA链
llm = ChatOllama(model="qwen2.5", temperature=0.0)  # 使用ollama模型
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs={"document_separator": "<<<<>>>>>"},
)

In [54]:
data[10]

Document(metadata={'source': '../data/product_data.csv', 'row': 10}, page_content="product_name: 高清电视机\ndescription: 规格:\n尺寸：50''。\n\n为什么我们热爱它:\n我们的高清电视机拥有出色的画质和强大的音效，带来沉浸式的观看体验。\n\n材质与护理:\n使用干布清洁。\n\n构造:\n由塑料、金属和电子元件制成。\n\n其他特性:\n支持网络连接，可以在线观看视频。\n配备遥控器。\n在韩国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。")

In [ ]:
data[11]

Document(metadata={'source': '../data/product_data.csv', 'row': 11}, page_content="product_name: 旅行背包\ndescription: 规格:\n尺寸：18'' x 12'' x 6''。\n\n为什么我们热爱它:\n我们的旅行背包拥有多个实用的内外袋，轻松装下您的必需品，是短途旅行的理想选择。\n\n材质与护理:\n可以手洗，自然晾干。\n\n构造:\n由防水尼龙制成。\n\n其他特性:\n附带可调节背带和安全锁。\n在中国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。")

In [56]:
examples = [
    {"query": "高清电视机怎么进行护理？", "answer": "使用干布清洁。"},
    {"query": "旅行背包有内外袋吗？", "answer": "有。"},
]
print(type(examples))
print(examples[0])

<class 'list'>
{'query': '高清电视机怎么进行护理？', 'answer': '使用干布清洁。'}


In [57]:
from langchain.evaluation.qa import (
    QAGenerateChain,
)  # 导入QA生成链，它将接收文档，并从每个文档中创建一个问题答案对

# 下面是langchain.evaluation.qa.generate_prompt中的源码，在template的最后加上“请使用中文输出”
from langchain.output_parsers.regex import RegexParser
from langchain.prompts import PromptTemplate
from langchain.base_language import BaseLanguageModel
from typing import Any

template = """You are a teacher coming up with questions to ask on a quiz. 
Given the following document, please generate a question and answer based on that document.

Example Format:
<Begin Document>
...
<End Document>
QUESTION: question here
ANSWER: answer here

These questions should be detailed and be based explicitly on information in the document. Begin!

<Begin Document>
{doc}
<End Document>
请使用中文输出
"""
output_parser = RegexParser(
    regex=r"QUESTION: (.*?)\nANSWER: (.*)", output_keys=["query", "answer"]
)
PROMPT = PromptTemplate(
    input_variables=["doc"], template=template, output_parser=output_parser
)


# 继承QAGenerateChain
class ChineseQAGenerateChain(QAGenerateChain):
    """LLM Chain specifically for generating examples for question answering."""

    @classmethod
    def from_llm(cls, llm: BaseLanguageModel, **kwargs: Any) -> QAGenerateChain:
        """Load QA Generate Chain from LLM."""
        return cls(llm=llm, prompt=PROMPT, **kwargs)


example_gen_chain = ChineseQAGenerateChain.from_llm(
    ChatOllama(model="qwen2.5", temperature=0.0)  # 使用ollama模型
)  # 通过传递 ollama 语言模型来创建这个链
new_examples = example_gen_chain.apply([{"doc": t} for t in data[:5]])

# 查看用例数据
new_examples

[{'qa_pairs': {'query': '这款全自动咖啡机有哪些尺寸选项？',
   'answer': "这款全自动咖啡机有两种尺寸选项：大型 - 尺寸为13.8'' x 17.3''；中型 - 尺寸为11.5'' x 15.2''。"}},
 {'qa_pairs': {'query': '根据文档中的描述，这款电动牙刷有哪些清洁模式和功能？',
   'answer': '文档中提到，这款电动牙刷具有多种清洁模式和其他特性，具体包括定时功能。'}},
 {'qa_pairs': {'query': '这款维生素C泡腾片每盒含有多少片？', 'answer': '每盒含有20片。'}},
 {'qa_pairs': {'query': '这款无线蓝牙耳机有哪些主要特点和规格？',
   'answer': '这款无线蓝牙耳机的主要特点包括：配备了降噪技术、长达8小时的电池续航力、快速充电功能以及内置麦克风支持接听电话。其规格方面，单个耳机尺寸为1.5英寸 x 1.3英寸，并且由耐用的塑料和金属构成，配备有软质耳塞。此外，清洁时只需用湿布擦拭即可。这款产品是在韩国制造的。'}},
 {'qa_pairs': {'query': '这款瑜伽垫的尺寸是多少？', 'answer': '这款瑜伽垫的尺寸是24英寸 x 68英寸。'}}]

In [58]:
print(new_examples[0])
print("\n")
print(data[0])

{'qa_pairs': {'query': '这款全自动咖啡机有哪些尺寸选项？', 'answer': "这款全自动咖啡机有两种尺寸选项：大型 - 尺寸为13.8'' x 17.3''；中型 - 尺寸为11.5'' x 15.2''。"}}


page_content='product_name: 全自动咖啡机
description: 规格:
大型 - 尺寸：13.8'' x 17.3''。
中型 - 尺寸：11.5'' x 15.2''。

为什么我们热爱它:
这款全自动咖啡机是爱好者的理想选择。 一键操作，即可研磨豆子并沏制出您喜爱的咖啡。它的耐用性和一致性使它成为家庭和办公室的理想选择。

材质与护理:
清洁时只需轻擦。

构造:
由高品质不锈钢制成。

其他特性:
内置研磨器和滤网。
预设多种咖啡模式。
在中国制造。

有问题？ 请随时联系我们的客户服务团队，他们会解答您的所有问题。' metadata={'source': '../data/product_data.csv', 'row': 0}


In [59]:
examples += [v for item in new_examples for k, v in item.items()]
qa.run(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


'您提到的产品并不是高清电视机，而是几个不同的家居用品：LED台灯、陶瓷保温杯、太阳能庭院灯和无线蓝牙耳机。根据提供的信息，这些产品并没有关于高清电视机的护理说明。\n\n如果您需要了解高清电视机的护理方法，请提供更具体的信息或描述，或者告诉我您想了解的是哪一款产品，这样我才能给出准确的回答。对于一般电子产品的清洁保养，通常建议使用柔软的干布轻轻擦拭屏幕和外壳，并避免使用含有酒精或其他化学溶剂的湿布，以防损坏设备。'

In [60]:
import langchain

langchain.debug = True

# 重新运行与上面相同的示例，可以看到它开始打印出更多的信息
qa.run(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "高清电视机怎么进行护理？"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "高清电视机怎么进行护理？",
  "context": "product_name: LED台灯\ndescription: 规格:\n尺寸：6'' x 6'' x 18''。\n\n为什么我们热爱它:\n我们的LED台灯亮度可调，采用节能LED灯珠，不仅可以提供明亮的光照，还可以节省能源。\n\n材质与护理:\n使用湿布擦拭。\n\n构造:\n由金属和塑料制成。\n\n其他特性:\n具有触控开关和调光功能。\n在中国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。<<<<>>>>>product_name: 陶瓷保温杯\ndescription: 规格:\n容量：350ml。\n\n为什么我们热爱它:\n我们的陶瓷保温杯设计优雅，保温效果好，是办公室和户外活动的理想伴侣。\n\n材质与护理:\n可以手洗或洗碗机清洗。\n\n构造:\n由高品质陶瓷和不锈钢制成。\n\n其他特性:\n有多种颜色可选。\n在中国制造。\n\n有问题？请随时联系我们的客户服务团队，他们会解答您的所有问题。<<<<>>>>>product_name: 太阳能庭院灯\ndescription: 规格:\n高度：18''。\n\n为什么我们热爱它:\n我们的太阳能庭院灯无需电源，只需将其放在阳光下即可充电。到了晚上，它会自动点亮，为您的庭院增添温馨的氛围。\n\n材质与护理:\n可以用湿布清洁。\n\n构造:\n由塑料和太阳能板制成。\n\n其他特性:\n自动感应，日落后自动点亮。\n在中国制造。\n\n有问题？请随时联系我们的客

'您提到的产品并不是高清电视机，而是几个不同的家居用品：LED台灯、陶瓷保温杯、太阳能庭院灯和无线蓝牙耳机。根据提供的信息，这些产品并没有关于高清电视机的护理说明。\n\n如果您需要了解高清电视机的护理方法，请提供更具体的信息或描述，或者告诉我您想了解的是哪一款产品，这样我才能给出准确的回答。对于一般电子产品的清洁保养，通常建议使用柔软的干布轻轻擦拭屏幕和外壳，并避免使用含有酒精或其他化学溶剂的湿布，以防损坏设备。'

In [63]:
langchain.debug = False

# 为所有不同的示例创建预测
predictions = qa.apply(examples)

# 对预测的结果进行评估，导入QA问题回答，评估链，通过语言模型创建此链
from langchain.evaluation.qa import QAEvalChain  # 导入QA问题回答，评估链

# 通过使用ollama模型进行评估
llm = ChatOllama(model="qwen2.5", temperature=0.0)
eval_chain = QAEvalChain.from_llm(llm)

# 在此链上调用evaluate，进行评估
graded_outputs = eval_chain.evaluate(examples, predictions)

/var/folders/hm/wyn_k9d108zfv2y674p1hv_r0000gn/T/ipykernel_83099/503304491.py:4: LangChainDeprecationWarning: The method `Chain.apply` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~batch` instead.
  predictions = qa.apply(examples)




> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [67]:
# 我们将传入示例和预测，得到一堆分级输出，循环遍历它们打印答案
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]["query"])
    print("Real Answer: " + examples[i]["answer"])
    print("Predicted Answer: " + predictions[i]["result"])
    print("Predicted Grade: " + graded_outputs[i]["results"])
    print()

Example 0:
Question: 高清电视机怎么进行护理？
Real Answer: 使用干布清洁。
Predicted Answer: 您提到的产品并不是高清电视机，而是几个不同的家居用品：LED台灯、陶瓷保温杯、太阳能庭院灯和无线蓝牙耳机。根据提供的信息，这些产品并没有关于高清电视机的护理说明。

如果您需要了解高清电视机的护理方法，请提供更具体的信息或描述，或者告诉我您想了解的是哪一款产品，这样我才能给出准确的回答。对于一般电子产品的清洁保养，通常建议使用柔软的干布轻轻擦拭屏幕和外壳，并避免使用含有酒精或其他化学溶剂的湿布，以防损坏设备。
Predicted Grade: GRADE: INCORRECT

解释：学生答案中提供了关于高清电视机护理的一般性建议，但没有直接回答如何进行护理。正确的答案是“使用干布清洁”，因此学生的回答与正确答案不符。不过，考虑到学生提供了一些相关的信息，并且没有包含错误信息，可以认为学生部分理解了问题的核心，但由于未能准确给出真确的答案，所以判定为错误。

Example 1:
Question: 旅行背包有内外袋吗？
Real Answer: 有。
Predicted Answer: 是的，旅行背包拥有多个实用的内外袋，这使得您可以轻松装下必需品。
Predicted Grade: GRADE: CORRECT

虽然学生的回答比正确答案多了关于旅行背包内部和外部都有袋子的信息，并且使用了更多的词汇来表达，但这些额外信息并没有与正确答案相矛盾。因此，可以认为学生答案是正确的。

Example 2:
Question: 这款全自动咖啡机有哪些尺寸选项？
Real Answer: 这款全自动咖啡机有两种尺寸选项：大型 - 尺寸为13.8'' x 17.3''；中型 - 尺寸为11.5'' x 15.2''。
Predicted Answer: 这款自动咖啡机的尺寸是12'' x 8'' x 14''。
Predicted Grade: GRADE: INCORRECT

解释：学生答案只提供了单一的尺寸信息，并且与真实答案中的两种尺寸选项不符。因此，该答案在事实准确性上是错误的。

Example 3:
Question: 根据文档中的描述，这款电动牙刷有哪些清洁模式和功能？
Real Answer: 文档中提到，这款电